## Square plate attached at the end of a pendulum

In [ ]:
from sympy import *
import sympy.physics.mechanics as me
me.init_vprinting()

In [ ]:
a, L, g, m_rod, m_plate = symbols("a, L, g, m, M")
q1, q2 = me.dynamicsymbols("q1, q2")
q1d, q2d = me.dynamicsymbols("q1, q2", 1)
omega1, omega2 = me.dynamicsymbols("omega_1, omega_2")

# reaction forces acting on point O
Rx, Ry = me.dynamicsymbols("R_x, R_y")
# auxiliary velocities needed to compute the reaction forces
u1_aux, u2_aux = me.dynamicsymbols("u_aux^((1:3))")

N, A, B = symbols("N, A, B", cls=me.ReferenceFrame)
A.orient_axis(N, N.z, q1)
B.orient_axis(N, N.z, q2)

O, P, G, C, R = symbols("O, P, G, C, R", cls=me.Point)
P.set_pos(O, -L * A.y)
G.set_pos(O, -L / 2 * A.y)
C.set_pos(P, a / 2 * (B.x - B.y))
R.set_pos(P, -a * B.y)

O.set_vel(N, u1_aux * N.x + u2_aux * N.y)
P.v2pt_theory(O, N, A)
P.a2pt_theory(O, N, A)
G.v2pt_theory(O, N, A)
G.a2pt_theory(O, N, A)
C.v2pt_theory(P, N, B)
C.a2pt_theory(P, N, B)
R.v2pt_theory(P, N, B)
R.a2pt_theory(P, N, B)

coordinates = [q1, q2]
velocities = [omega1, omega2]
kde = [q1d - omega1, q2d - omega2]
kane = me.KanesMethod(
    N,
    q_ind=coordinates, 
    u_ind=velocities,
    kd_eqs=kde,
    u_auxiliary=[u1_aux, u2_aux]
)

Number of operations to compute the DCM from B to N:

In [ ]:
dcm_B_N = B.dcm(N)
display(
    dcm_B_N,
    count_ops(dcm_B_N),
    count_ops(dcm_B_N, visual=True)
)

In [ ]:
id_rod = me.inertia(A, m_rod * L**2 / 12, 0, m_rod * L**2 / 12)
id_plate = me.inertia(B, m_plate * a**2 / 12, m_plate * a**2 / 12, m_plate * a**2 / 6)
inertia_rod = (id_rod, G)
inertia_plate = (id_plate, C)

rod = me.RigidBody("rod", G, A, m_rod, inertia_rod)
plate = me.RigidBody("plate", C, B, m_plate, inertia_plate)
bodies = [rod, plate]

loads = [
    (G, -m_rod * g * N.y),
    (C, -m_plate * g * N.y),
    (O, Rx * N.x + Ry * N.y)
]

fr, frstar = kane.kanes_equations(bodies, loads)
trigsimp(fr + frstar)

### Numerical results

In [ ]:
import numpy as np
from pydy.system import System

sys = System(kane)
sys.constants = {
    m_rod: 0,
    m_plate: 10,
    a: 6,
    L: 12,
    g: 9.81
}
sys.initial_conditions = {
    q1: 0, q2: 0,
    omega1: 0, omega2: 0
}

fps = 60
t0, tf = 0, 15
n = int(fps * (tf - t0))
sys.times = np.linspace(t0, tf, n)

results = sys.integrate()

In [ ]:
import matplotlib.pyplot as plt

results_deg = np.rad2deg(results)
fig, ax = plt.subplots(2, 1, sharex=True)
ax[0].plot(sys.times, results_deg[:, 0], label="$q_{1}$")
ax[0].plot(sys.times, results_deg[:, 1], label="$q_{2}$")
ax[0].set_ylabel(r"Angle [deg]")
ax[0].legend()
ax[1].plot(sys.times, results_deg[:, 2], label=r"$\omega_{1}$")
ax[1].plot(sys.times, results_deg[:, 3], label=r"$\omega_{2}$")
ax[1].set_ylabel(r"Velocity [deg / s]")
ax[1].set_xlabel("Time [s]")
ax[1].legend()
plt.show()

### Compute reaction forces at O

In [ ]:
R_eqns = solve(kane.auxiliary_eqs, [Rx, Ry])
R_eqns = Matrix(list(R_eqns.values())).T
R_eqns.T

In [ ]:
t = symbols("t")
arguments_1 = [q1, q2, omega1, omega2] + list(sys.constants.keys())
arguments_2 = (
    [q1, q2, omega1, omega2] +
    [q1d, q2d, omega1.diff(t), omega2.diff(t)] +
    list(sys.constants.keys())
)

mm_func = lambdify(arguments_1, kane.mass_matrix_full)
fm_func = lambdify(arguments_1, kane.forcing_full)
R_eqns_func = lambdify(arguments_2, R_eqns)

states_dot = np.zeros_like(results)
reactions = np.zeros((results.shape[0], 2))
for i in range(results.shape[0]):
    args_1 = list(results[i, :]) + list(sys.constants.values())
    states_dot[i, :] = np.linalg.solve(
        mm_func(*args_1),
        fm_func(*args_1)
    ).flatten()
    args_2 = list(results[i, :]) + list(states_dot[i, :]) + list(sys.constants.values())
    reactions[i, :] = R_eqns_func(*args_2)

fig, ax = plt.subplots()
ax.plot(sys.times, reactions[:, 0], label="$R_{x}$")
ax.plot(sys.times, reactions[:, 1], label="$R_{y}$")
ax.set_xlabel("time [s]")
ax.set_ylabel("force [N]")
ax.legend()
plt.show()

### Compute velocity of point P

In [ ]:
v_P = P.vel(N).to_matrix(N)
kindiffdict = kane.kindiffdict()
aux_dict = {aux_speed: 0 for aux_speed in [u1_aux, u2_aux]}
v_P = v_P.subs(kindiffdict).subs(aux_dict)
v_P

In [ ]:
v_P_x_func = lambdify([L, q1, q2, omega1, omega2], v_P[0])
v_P_y_func = lambdify([L, q1, q2, omega1, omega2], v_P[1])
v_P_x_res = v_P_x_func(sys.constants[L], results[:, 0], results[:, 1], results[:, 2], results[:, 3])
v_P_y_res = v_P_y_func(sys.constants[L], results[:, 0], results[:, 1], results[:, 2], results[:, 3])

fig, ax = plt.subplots()
ax.plot(sys.times, v_P_x_res, label=r"$\vec{v}_{P} \cdot \hat{n}_{x}$")
ax.plot(sys.times, v_P_y_res, label=r"$\vec{v}_{P} \cdot \hat{n}_{y}$")
ax.legend()
ax.set_xlabel("Time [s]")
ax.set_ylabel("Velocity [m/s]")
plt.show()

### Animation with PyDy

In [ ]:
from pydy.viz import Scene, Cylinder, Box, VisualizationFrame

rod_geom = Cylinder(length=L, radius=0.5, color='grey', name="rod")
plate_geom = Box(a, a, 0.5, color="orange", name="plate")

rod_vf = VisualizationFrame('rod vf', A, G, rod_geom)
plate_vf = VisualizationFrame('plate vf', B, C, plate_geom)

# arguments to Scene: reference frame, origin, visual frames, system
scene = Scene(N, O, rod_vf, plate_vf, system=sys)
scene.display_jupyter(axes_arrow_length=5)

### Animation with Matplotlib

In [ ]:
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

fig, ax = plt.subplots()

def get_line_coords(results, i):
    q1_val = results[i, 0]
    L_val = sys.constants[L]
    y = [0, -L_val * np.cos(q1_val)]
    x = [0, L_val * np.sin(q1_val)]
    return x, y

R = lambda angle: np.array([
    [np.cos(angle), np.sin(angle), 0],
    [-np.sin(angle), np.cos(angle), 0],
    [0, 0, 1],
])

standard_plate = np.array([
    [0, 0, 1],
    [1, 0, 1],
    [1, 1, 1],
    [0, 1, 1],
    [0, 0, 1],
])

def get_plate_coords(results, i):
    q1_val = results[i, 0]
    q2_val = results[i, 1]
    a_val = sys.constants[a]
    L_val = sys.constants[L]
    rotated_plate = (a_val * standard_plate) @ R(q2_val)
    y = -L_val * np.cos(q1_val) - rotated_plate[:, 0]
    x = L_val * np.sin(q1_val) + rotated_plate[:, 1]
    return x, y

idx = 0
line, = ax.plot(*get_line_coords(results, idx))
plate, = ax.plot(*get_plate_coords(results, idx))
ax.set_aspect("equal")
ax.axis([-15, 15, -25, 5])

def update(idx):
    line.set_data(*get_line_coords(results, idx))
    plate.set_data(*get_plate_coords(results, idx))
    ax.set_title("Time = {:.2f} s".format(sys.times[idx]))

ani = FuncAnimation(fig, func=update, frames=len(sys.times))
HTML(ani.to_jshtml(fps=fps))
# ani.save("pendulum-square-plate.mp4", fps=60)

## With the Joints Framework

In [ ]:
from sympy import *
import sympy.physics.mechanics as me
import numpy as np
from pydy.system import System
from pydy.viz import Scene, Cylinder, Box, VisualizationFrame, Sphere
me.init_vprinting()

a, L, g, m_rod, m_plate = symbols("a, L, g, m, M")
q1, q2 = me.dynamicsymbols("q1, q2")
omega1, omega2 = me.dynamicsymbols("omega_1, omega_2")
M, N, A, B = symbols("M, N, A, B", cls=me.ReferenceFrame)
N.orient_axis(M, M.z, pi)

G, C = symbols("G, C", cls=me.Point)

wall = me.RigidBody("Wall", frame=N, mass=0)
rod = me.RigidBody(
    "Rod", G, A, m_rod,
    (me.inertia(A, 0, 0, m_rod * L**2 / 12), G)
)
plate = me.RigidBody(
    "Plate", C, B, m_plate,
    (me.inertia(B, m_plate * a**2 / 12, m_plate * a**2 / 12, m_plate * a**2 / 6), C)
)

system = me.System.from_newtonian(wall)
system.add_joints(
    me.PinJoint(
        "Joint 1", wall, rod, coordinates=q1, speeds=omega1,
        child_point=-L / 2 * rod.frame.y,
        joint_axis=wall.frame.z
    ),
    me.PinJoint(
        "Joint 2", rod, plate, coordinates=q2, speeds=omega2,
        parent_point=L / 2 * rod.frame.y,
        child_point=a / 2 * (B.x - B.y),
        joint_axis=rod.frame.z
    )
)

system.apply_uniform_gravity(g * system.y)

system.validate_system()
eoms = system.form_eoms()
eoms

In [ ]:
dcm_B_N = B.dcm(N)
display(
    dcm_B_N,
    count_ops(dcm_B_N),
    count_ops(dcm_B_N, visual=True)
)

In [ ]:
sys = System(kane)
sys.constants = {
    m_rod: 0,
    m_plate: 10,
    a: 6,
    L: 12,
    g: 9.81
}
sys.initial_conditions = {
    q1: 0, q2: pi / 2,
    omega1: 0, omega2: 0
}

fps = 60
t0, tf = 0, 15
n = int(fps * (tf - t0))
sys.times = np.linspace(t0, tf, n)

results = sys.integrate()

rod_geom = Cylinder(length=L, radius=0.5, color='grey', name="rod")
plate_geom = Box(a, a, 0.5, color="orange", name="plate")

rod_vf = VisualizationFrame('rod vf', A, rod.masscenter, rod_geom)
plate_vf = VisualizationFrame('plate vf', B, plate.masscenter, plate_geom)

# arguments to Scene: reference frame, origin, visual frames, system
O = rod.masscenter.locatenew("O", -L/2 * rod.frame.y)
scene = Scene(M, O, rod_vf, plate_vf, system=sys)
scene.display_jupyter(axes_arrow_length=5)

In [ ]:
B.dcm(N)

In [ ]:
count_ops(B.dcm(N))

In [ ]:
count_ops(B.dcm(N), visual=True)